In [18]:
import pandas as pd
import pingouin as pg

In [19]:
df = pd.read_csv("football.csv")

In [20]:
df.head()

,Key,Team,League,Season,Rank,Games,Wins,Draws,Losses,Points,...,Nutmegs,Controlled,DistMovedWithBall,ProgressiveDistMoved,ProgC,ProgressiveIntoFinalThird,ProgressiveInto18Yard,Miscontrols,MiscontrolsAfterTackle,ProgressivePassReceived
0,Chelsea DL 2009/2010,Chelsea,Premier League,2009/2010,1.0,38.0,27.0,5.0,6.0,86.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Manchester United DL 2009/2010,Manchester United,Premier League,2009/2010,2.0,38.0,27.0,4.0,7.0,85.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tottenham DL 2009/2010,Tottenham,Premier League,2009/2010,4.0,38.0,21.0,7.0,10.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arsenal DL 2009/2010,Arsenal,Premier League,2009/2010,3.0,38.0,23.0,6.0,9.0,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aston Villa DL 2009/2010,Aston Villa,Premier League,2009/2010,6.0,38.0,17.0,13.0,8.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Add start and end of the Season

In [21]:
df["Start Season"] = df["Season"].apply(lambda x : int(x[:4]))
df["End Season"] = df["Season"].apply(lambda x : (x[5:]))

In [22]:
pl = df[(df["League"] == "Premier League") & (df["Rank"] == 1)][["League","Team","Start Season", "End Season","Points","Wins"]].reset_index(drop = True)

In [23]:
ll = df[(df["League"] == "La Liga") & (df["Rank"] == 1)][["League","Team","Start Season", "End Season","Points","Wins"]].reset_index(drop = True)

In [38]:
pl["Points"].mean()

89.76923076923077

In [37]:
ll["Points"].mean()

92.46153846153847

# Check normality

In [33]:
print(f'''Premier League Winner Points in 2009-2022 are normal -> {pg.normality(pl.Points.values)["normal"].values[0]}''')
print(f'''LaLiga Winner Points in 2009-2022 are normal -> {pg.normality(ll.Points.values)["normal"].values[0]}''')

Premier League Winner Points in 2009-2022 are normal -> True
LaLiga Winner Points in 2009-2022 are normal -> True


# T-Test Points

## 2009 - 2022

In [26]:
pg.ttest(ll.Points.values, pl.Points.values, correction = False, alternative = 'greater')

,T,dof,alternative,p-val,CI95%,cohen-d,BF10,power
T-test,1.170357,24,greater,0.126674,"[-1.24, inf]",0.459052,1.198,0.305916
